In [0]:
!pip install openpyxl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.9/250.9 kB 6.4 MB/s eta 0:00:00

[notice] A new release of pip available: 22.2.2 -> 24.2
[notice] To update, run: pip install --upgrade pip


In [0]:
import pandas as pd
database_name = "data_pipelines_tutorial"
URL = 'https://www2.census.gov/programs-surveys/international-programs/tables/time-series/prh/kenya.xlsx'

df_raw = pd.read_excel(URL, sheet_name='2000 - 2040', skiprows=2, header=None)
# Read correct header
header = df_raw.iloc[1]
df_raw.columns = header
df_raw = df_raw.drop([0,1,2])

# Extract Total population columns 
df_pop_wide = df_raw[df_raw.ADM_LEVEL==1][['CNTRY_NAME', 'ADM1_NAME']+[x for x in header if 'BTOTL' in x]]
df_pop = pd.melt(df_pop_wide, id_vars=['CNTRY_NAME', 'ADM1_NAME'], var_name='year', value_name='population')
df_pop['year'] = df_pop['year'].str.extract(r'(\d+)').astype(int)
df_pop.columns = ['country_name', 'adm1_name', 'year', 'population']

# Modifications to the admin1 and county name and add data_source
name_standardization = {
    ''
}
df_pop['country_name'] = df_pop['country_name'].str.title()
df_pop['adm1_name'] = df_pop['adm1_name'].str.replace(r'[-/]+', ' ', regex=True).str.title()
df_pop = df_pop.astype({'year': 'int', 'population': 'int'})
df_pop = df_pop.sort_values(['adm1_name', 'year'], ignore_index=True)

# Save to subnational population table
spark.createDataFrame(df_pop).write.format("delta").mode("overwrite").saveAsTable(f"{database_name}.kenya_subnational_population")